In [ ]:
import numpy as np
import cv2
import pathlib

import tensorflow as tf
import matplotlib.pyplot as plt

#data obtained from https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data

In [ ]:
training_image_dir = "./Pneumonia_dataset/chest_xray/train"
test_img_dir = "./Pneumonia_dataset/chest_xray/test"
val_img_dir = "./Pneumonia_dataset/chest_xray/val"

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

train_iter = datagen.flow_from_directory(training_image_dir, batch_size = 8, target_size = (1024,1024))
val_iter = datagen.flow_from_directory(val_img_dir, batch_size=8, target_size=(1024, 1024))
test_iter = datagen.flow_from_directory(test_img_dir, batch_size=8, target_size=(1024, 1024))

train_images, train_labels = next(train_iter)
test_images, test_labels = next(test_iter)
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_images.shape, train_images.min(), train_images.max()))

In [ ]:
def build_cnn():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', strides=(2,2), input_shape=(1024, 1024, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), strides=(2,2),activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), strides=(2,2),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    return model

model = build_cnn()

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_iter, 
                    epochs=100,
                    validation_data=test_iter, 
                    validation_steps=32)

In [ ]:
print(history.history.keys())

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy') 
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')

plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print(test_acc)